#### 環境の確認

以降では、Pythonの機械学習用ライブラリーであるscikit-learnを利用するため、
バージョンを確認しておきます。

0.21以上であれば、以降のプログラムを実行することが可能なはずです。

In [1]:
 # バージョンの確認
import sklearn
print(sklearn.__name__, sklearn.__version__)
import numpy as np
print(np.__name__, np.__version__)

sklearn 1.0.2
numpy 1.22.1


## 2.4 データの加工(第1ラウンド)
ここでは上記の仮説に従って、テキストデータから特徴を抽出します。\
具体的には、テキストに現れる名詞・形容詞・動詞・副詞の全単語について出現回数を求め、表形式にまとめます。

実際に以下の手順に従って特徴抽出を行いましょう。
1. 原文テキストの抽出\
テキストデータから余分な要素を取り除き**原文テキストを復元**します。
1. 形態素解析\
作品毎に形態素解析を行うことでテキストを単語に分解します。
2. 名詞・形容詞・動詞・副詞の抽出
3. 名詞・形容詞・動詞・副詞の数え上げ

まず、以下のセルを実行して、dataフォルダが作成されているか確認して下さい。


In [2]:
mkdir data

mkdir: data: File exists


まず、以下のセルを実行して、dataフォルダにテキストファイルがダウンロード済みであるかを確認して下さい。
例えば、`agnino_kami.txt`、`ahen.txt`などのファイルがない場合は、


In [3]:
ls *.txt

zsh:1: no matches found: *.txt


---
#### 1.原文テキストの抽出
まずステップ1では、原文テキストの抽出を行います。


青空文庫からダウンロードしたテキストファイルには、原文テキストにはない要素が含まれています。

```
\u3000 ある日の暮方の事である。一人の下人《げにん》が、 羅生門《らしょうもん》の下で雨やみを待っていた。\n\u3000 広い門の下には、この男のほかに誰もいない。ただ、所々|丹塗《にぬり》の剥《は》げた、大きな円柱《まるばしら》に、 蟋蟀《きりぎりす》が一匹とまっている。
...(中略)... 
  下人はとうとう、老婆の腕をつかんで、無理にそこへ※［＃「てへん＋丑」、第4水準2-12-93］《ね》じ倒した。丁度、鶏《にわとり》の脚のような、骨と皮ばかりの腕である。\n「何をしていた。云え。云わぬと、これだぞよ。」
  \n\u3000下人は、老婆をつき放すと、いきなり、太刀の鞘《さや》を払って、白い鋼《はがね》の色をその眼の前へつきつけた。
...(中略)... 
そうし て、そこから、短い白髪《しらが》を倒《さかさま》にして、 門の下を覗きこんだ。外には、ただ、黒洞々《こくとうとう》 たる夜があるばかりである。\n\u3000 下人の行方《ゆくえ》 は、誰も知らない。
```

>- ユニコードエスケープは、UTF-8やUTF-16で符号化された文字を 直接指定する方法で、例えば、\u3000 は全角のスペースを表します。
>- \nは改行を表す制御文字です。
>- 《げにん》などの漢字の読みの表示は、青空文庫で決めている書式です。
>- JIS水準が高い漢字は、注釈記号を使って表示されます。
例えば、※\[#「てへん+丑」、第 4 水準 2-12-93\]などです。
>- 注釈や漢文の返り点なども注釈記号を用いて記述されます。注釈記号は青空文庫で決めている書式です。

これらの要素は形態素解析の結果に影響を及ぼす可能性のあるものがあります。

>- ユニコードエスケープと制御記号は標準で定められた書式で、形態素解析に影響を与えないので、ここでは除去しません。
>- ルビや注釈は青空文庫固有の書式で記述されているため形態素解析を実行する上で障害となる可能性があります。JIS水準が高い漢字を含む語は読みで置き換え、その他の注釈とルビは削除します。

In [4]:
import re # 正規表現操作に使用するモジュール

kosuijun = re.compile(u'([一-龥]*※［[^］]+］)+[一-龥]*《([^》]+)》') # 高水準漢字のパターン
ruby = re.compile(r'《[^》]+》') # ルビのパターン
chu = re.compile(r'［[^］]+］') # 注釈のパターン

text = "下人はとうとう、老婆の腕をつかんで、無理にそこへ※［＃「てへん＋丑」、第4水準2-12-93］《ね》じ倒した。丁度、鶏《にわとり》の脚のような、骨と皮ばかりの腕である。\n「何をしていた。云え。云わぬと、これだぞよ。」\n\u3000下人は、老婆をつき放すと、いきなり、太刀の鞘《さや》を払って、白い鋼《はがね》の色をその眼の前へつきつけた。"
text_mod = re.sub(kosuijun, r'\2',text) # 注釈の削除
text_mod = re.sub(ruby,"", text_mod) # ルビの削除
text_mod = re.sub(chu,"",text_mod) # 注釈の削除

print(f"[元の文章]\n{text}\n")
print(f"[削除後]\n{text_mod}")

[元の文章]
下人はとうとう、老婆の腕をつかんで、無理にそこへ※［＃「てへん＋丑」、第4水準2-12-93］《ね》じ倒した。丁度、鶏《にわとり》の脚のような、骨と皮ばかりの腕である。
「何をしていた。云え。云わぬと、これだぞよ。」
　下人は、老婆をつき放すと、いきなり、太刀の鞘《さや》を払って、白い鋼《はがね》の色をその眼の前へつきつけた。

[削除後]
下人はとうとう、老婆の腕をつかんで、無理にそこへねじ倒した。丁度、鶏の脚のような、骨と皮ばかりの腕である。
「何をしていた。云え。云わぬと、これだぞよ。」
　下人は、老婆をつき放すと、いきなり、太刀の鞘を払って、白い鋼の色をその眼の前へつきつけた。


In [5]:
text = "僕はこの頃漫然と兪※［＃「木＋越」、第3水準1-86-11］《ゆゑつ》の「右台仙館筆記《うたいせんくわんひつき》」を読んでゐるうちにかう云ふ俗伝は支那人の中にもあつたと云ふことを発見した。それは同書の中に掲げた「賈慎庵《かしんあん》」の話に出合つたからである。\n\u3000賈慎庵は何でも乾隆《けんりゆう》の末の老諸生の一人だつたと云ふことである。それが或夜の夢の中に大きい役所らしい家の前へ行つた。家は重門｜尽《ことごと》く掩《おほ》ひ、闃《げき》としてどこにも人かげは見えない。「正に徘［＃「徘」は底本では「俳」］徊《はいくわい》の間、俄《には》かに数人あり、一婦を擁して遠きより来り、この門の外に至る。」"
text_mod = re.sub(kosuijun, r'\2', text) # ルビの削除
text_mod = re.sub(ruby,"", text_mod) # ルビの削除
text_mod = re.sub(chu,"", text_mod) # 注釈の削除
print(f"[元の文章]\n{text}\n")
print(f"[削除後]\n{text_mod}")

[元の文章]
僕はこの頃漫然と兪※［＃「木＋越」、第3水準1-86-11］《ゆゑつ》の「右台仙館筆記《うたいせんくわんひつき》」を読んでゐるうちにかう云ふ俗伝は支那人の中にもあつたと云ふことを発見した。それは同書の中に掲げた「賈慎庵《かしんあん》」の話に出合つたからである。
　賈慎庵は何でも乾隆《けんりゆう》の末の老諸生の一人だつたと云ふことである。それが或夜の夢の中に大きい役所らしい家の前へ行つた。家は重門｜尽《ことごと》く掩《おほ》ひ、闃《げき》としてどこにも人かげは見えない。「正に徘［＃「徘」は底本では「俳」］徊《はいくわい》の間、俄《には》かに数人あり、一婦を擁して遠きより来り、この門の外に至る。」

[削除後]
僕はこの頃漫然とゆゑつの「右台仙館筆記」を読んでゐるうちにかう云ふ俗伝は支那人の中にもあつたと云ふことを発見した。それは同書の中に掲げた「賈慎庵」の話に出合つたからである。
　賈慎庵は何でも乾隆の末の老諸生の一人だつたと云ふことである。それが或夜の夢の中に大きい役所らしい家の前へ行つた。家は重門｜尽く掩ひ、闃としてどこにも人かげは見えない。「正に徘徊の間、俄かに数人あり、一婦を擁して遠きより来り、この門の外に至る。」


以上の方法で、原文を復元することができそうです。


#### 2.形態素解析
次にステップ2では、任意の文の形態素解析を実装する方法を紹介します。今回はMeCabを利用します。

日本語の文章に形態素解析をかけると、以下の分析が行われます。
- 文字列を形態素（単語）に分解
- 各形態素の品詞を特定
- 活用形を標準形（終止形）に変換


(以下は形態素解析について、本書からの引用です。)
> 文中で意味をもつ文字の並びの最小単位を、自然言語処理では**形態素**と呼ぶ。
単純のため単語と同義と考えてもよいが、
複数の単語が結合して別の単語を構成することがある一方、
形態素はそれ以上分解できない最小の単位を表す。
テキストの文法構造を明らかにするためには、
まず、単なる記号の列であるテキストを、形態素の集まりに整理し、
それぞれの形態素に対して、その品詞と用法を特定することが必要である。
このような処理のことを、**形態素解析**と呼ぶ。


#### MeCab
MeCabは広く使われている形態素解析のツールの一つです。品詞の特定に加えて、語の用法も特定することができます。

まず、MeCabと辞書をインストールします。
今回使用する辞書"ipadic"はMecabに同梱されていたIPAの辞書です。


In [6]:
pip install mecab-python3 ipadic

Note: you may need to restart the kernel to use updated packages.


試しに、以下の例文を形態素解析してみましょう。\
例文「一人の下人が、羅生門の下で雨やみを待っていた。」

In [7]:
 # 例文の形態素解析
import MeCab as mc
import ipadic

trg = mc.Tagger(ipadic.MECAB_ARGS)
node = trg.parseToNode("一人の下人が、羅生門の下で雨やみを待っていた。") # 例文
output = []
while node:
    temp = node.feature.split(",")
    output.append(":".join([temp[6],temp[0],temp[1]]))
    node = node.next
print("   ".join(output))

*:BOS/EOS:*   一:名詞:数   人:名詞:接尾   の:助詞:連体化   下人:名詞:一般   が:助詞:格助詞   、:記号:読点   羅生門:名詞:固有名詞   の:助詞:連体化   下:名詞:一般   で:助詞:格助詞   雨:名詞:一般   やみ:名詞:一般   を:助詞:格助詞   待つ:動詞:自立   て:助詞:接続助詞   いる:動詞:非自立   た:助動詞:*   。:記号:句点   *:BOS/EOS:*


以上から、任意の文で形態素解析を実装する方法がわかりました。

#### 3. 形容詞・動詞・名詞・副詞の抽出

ステップ3では任意の文から形容詞、動詞、名詞、副詞のみを取り出す関数を作成します。

In [8]:
def decomp(text): # 形容詞、動詞、名詞、副詞のみを取り出す
    trg = mc.Tagger(ipadic.MECAB_ARGS)
    node = trg.parseToNode(text)
    output = []
    while node:
        temp = node.feature.split(",")
        if temp[0] in ["形容詞", "動詞", "名詞", "副詞"]:
            output.append(":".join([temp[6],temp[0],temp[1]]))
        node = node.next
        if node is None:
            break
    return output

text = "一人の下人が、羅生門の下で雨やみを待っていた。"
print(decomp(text))
print("   ".join(decomp(text)))

['一:名詞:数', '人:名詞:接尾', '下人:名詞:一般', '羅生門:名詞:固有名詞', '下:名詞:一般', '雨:名詞:一般', 'やみ:名詞:一般', '待つ:動詞:自立', 'いる:動詞:非自立']
一:名詞:数   人:名詞:接尾   下人:名詞:一般   羅生門:名詞:固有名詞   下:名詞:一般   雨:名詞:一般   やみ:名詞:一般   待つ:動詞:自立   いる:動詞:非自立


別の例文でも試してみましょう。

In [9]:
 # 別の例文
print("   ".join(decomp("広い門の下には、この男のほかに誰もいない。")))

広い:形容詞:自立   門:名詞:一般   下:名詞:一般   男:名詞:一般   ほか:名詞:副詞可能   誰:名詞:代名詞   いる:動詞:自立


In [10]:
 # さらに別の例文
print("   ".join(decomp("ただ、所々｜丹塗の剥げた、大きな円柱に、蟋蟀が一匹とまっている。")))

所々:名詞:副詞可能   丹塗:名詞:一般   剥げる:動詞:自立   円柱:名詞:一般   蟋蟀:名詞:一般   一:名詞:数   匹:名詞:接尾   とまる:動詞:自立   いる:動詞:非自立


以上で、任意の文から名詞・形容詞・動詞・副詞のみをリストとして抽出する関数を作成することができました。

#### 4. 形容詞・動詞・名詞・副詞の数え上げ

ステップ3までで、任意のテキストのルビと注釈を削除し、全ての形態素を含むリストを作成しました。ステップ4では、各形態素がテキスト中に何回出現するかを数え上げます。



In [11]:
 # Counterを使用して理解を深める
from collections import Counter
text = "私の兄は私より背が高いです。あなたの弟は私よりも背が高いですか?"
print("   ".join(decomp(text))) # ステップ3までの結果

counter = Counter(decomp(text))
print(f"\n{counter}")
print(counter.most_common()) # (要素, 出現回数)というタプルを出現回数順に並べたリスト

私:名詞:代名詞   兄:名詞:一般   私:名詞:代名詞   背:名詞:一般   高い:形容詞:自立   あなた:名詞:代名詞   弟:名詞:一般   私:名詞:代名詞   背:名詞:一般   高い:形容詞:自立

Counter({'私:名詞:代名詞': 3, '背:名詞:一般': 2, '高い:形容詞:自立': 2, '兄:名詞:一般': 1, 'あなた:名詞:代名詞': 1, '弟:名詞:一般': 1})
[('私:名詞:代名詞', 3), ('背:名詞:一般', 2), ('高い:形容詞:自立', 2), ('兄:名詞:一般', 1), ('あなた:名詞:代名詞', 1), ('弟:名詞:一般', 1)]


---

### 40作品に対して特徴を抽出


では実際に、以上の4つのステップに従い、ダウンロードした芥川龍之介・菊池寛の各テキストデータに対してデータの加工を行います。

(データ加工の手順の再掲)
1. JIS水準の高い漢字を読みでおきかえ、ルビと注釈を取り除きます。
1. 作品毎に形態素解析を行うことでテキストを単語に分割します。
1. 名詞・形容詞・動詞・副詞を抽出します。
1. 名詞・形容詞・動詞・副詞を数え上げます。

まず、ステップ3までを実装し、その結果を[作品名].txt-decomp.txtという名前のテキストファイルとして書き出します。例えば、rashomon.text-decomp.txtの内容は以下の通りです。(長すぎるので一部を抜粋)

>日:名詞:非自立 暮方:名詞:副詞可能 事:名詞:非自立 一:名詞:数 人:名詞:接尾 下人:名詞:一般 羅生門:名詞:固有名詞 下:名詞:一般 雨:名詞:一般 やみ:名詞:一般 待つ:動詞:自立 いる:動詞:非自立 広い:形容詞:自立 門:名詞:一般 下:名詞:一般 男:名詞:一般 ほか:名詞:副詞可能 誰:名詞:代名詞 いる:動詞:自立 所々:名詞:副詞可能 丹塗:名詞:一般 剥げる:動詞:自立 円柱:名詞:一般 蟋蟀:名詞:一般 一:名詞:数 匹:名詞:接尾 とまる:動詞:自立 いる:動詞:非自立 羅生門:名詞:固有名詞 朱雀:名詞:固有名詞 大路:名詞:一般 ある:動詞:自立 以上:名詞:非自立 男:名詞:一般 ほか:名詞:副詞可能 雨:名詞:一般 やみ:名詞:一般 する:動詞:自立 市:名詞:一般 女:名詞:一般 笠:名詞:一般 *:名詞:一般 烏帽子:名詞:一般 もう:副詞:一般 二:名詞:数 三:名詞:数 人:名詞:接尾 ある:動詞:自立 そう:名詞:接尾 もの:名詞:非自立 それ:名詞:代名詞 男:名詞:一般 
>


In [12]:
 # 40編の作品のルビ・注釈を削除&名詞・形容詞・動詞・副詞のみを抽出
import re

kosuijun = re.compile(u'([一-龥]*※［[^］]+］)+[一-龥]*《([^》]+)》') # 高水準漢字のパターン
ruby = re.compile(r'《[^》]+》') # ルビのパターン
chu = re.compile(r'［[^］]+］') # 注釈のパターン

path = "data/"
akutagawa = ["rashomon.txt","yoba.txt","yabuno_naka.txt","mujina.txt","hana.txt",
            "haguruma.txt","torokko.txt","toshishun.txt","shunkan.txt","shujuno_kotoba.txt",
            "jashumon.txt","shogun.txt","shigo.txt","agunino_kami.txt","aruhino_oishi_kuranosuke.txt",
            "ogin.txt","ojigi.txt","kappa.txt","kiseru.txt","kumono_ito.txt"]
kikuchi = ["anegawano_kassen.txt","aru_koino_hanashi.txt","irefuda.txt","emukoshakuto_shashinshi.txt",
          "okujono_kyojin.txt","onshuno_kanatani.txt","onnagoto.txt","on'o_kaesu_hanashi.txt",
          "gimin_jinbee.txt","kunshoo_morau_hanashi.txt","gokuraku.txt","shusse.txt",
          "shobugoto.txt","dairiki_monogatari.txt","tojurono_koi.txt","aru_kogisho.txt",
          "minage_kyujogyo.txt","mumeisakkano_nikki.txt","wakasugi_saibancho.txt","bugyoto_ninsogaku.txt"]

for ds in akutagawa:
    text = open(path+ds,"r",encoding = "shift-jis").read()
    text = re.sub(kosuijun,r'\2',text) # 高水準漢字を読みに置き換え
    text = re.sub(ruby,"",text) # ルビの削除
    text = re.sub(chu,"",text) # 注釈の削除
    out = open(path+ds+"-decomp.txt", "w")
    out.write(" ".join(decomp(text)))
    out.close()
    
for ds in kikuchi:
    text = open(path+ds,"r",encoding = "shift-jis").read()
    text = re.sub(kosuijun,r'\2',text) # 高水準漢字を読みに置き換え
    text = re.sub(ruby,"", text) # ルビの削除
    text = re.sub(chu,"",text) # 注釈の削除
    out = open(path+ds+"-decomp.txt", "w")
    out.write(" ".join(decomp(text)))
    out.close()
    

print(open(path+"rashomon.txt-decomp.txt", "r").read())

日:名詞:非自立 暮方:名詞:副詞可能 事:名詞:非自立 一:名詞:数 人:名詞:接尾 下人:名詞:一般 羅生門:名詞:固有名詞 下:名詞:一般 雨:名詞:一般 やみ:名詞:一般 待つ:動詞:自立 いる:動詞:非自立 広い:形容詞:自立 門:名詞:一般 下:名詞:一般 男:名詞:一般 ほか:名詞:副詞可能 誰:名詞:代名詞 いる:動詞:自立 所々:名詞:副詞可能 丹塗:名詞:一般 剥げる:動詞:自立 円柱:名詞:一般 蟋蟀:名詞:一般 一:名詞:数 匹:名詞:接尾 とまる:動詞:自立 いる:動詞:非自立 羅生門:名詞:固有名詞 朱雀:名詞:固有名詞 大路:名詞:一般 ある:動詞:自立 以上:名詞:非自立 男:名詞:一般 ほか:名詞:副詞可能 雨:名詞:一般 やみ:名詞:一般 する:動詞:自立 市:名詞:一般 女:名詞:一般 笠:名詞:一般 *:名詞:一般 烏帽子:名詞:一般 もう:副詞:一般 二:名詞:数 三:名詞:数 人:名詞:接尾 ある:動詞:自立 そう:名詞:接尾 もの:名詞:非自立 それ:名詞:代名詞 男:名詞:一般 ほか:名詞:副詞可能 誰:名詞:代名詞 いる:動詞:自立 何故:副詞:助詞類接続 云う:動詞:自立 二:名詞:数 三:名詞:数 年:名詞:接尾 京都:名詞:固有名詞 地震:名詞:一般 辻:名詞:固有名詞 風:名詞:接尾 火事:名詞:一般 饑饉:名詞:一般 云う:動詞:自立 災:名詞:一般 つづく:動詞:自立 起る:動詞:自立 洛中:名詞:一般 さびれる:動詞:自立 方:名詞:接尾 一:名詞:数 通り:名詞:接尾 旧記:名詞:一般 よる:動詞:自立 仏像:名詞:一般 仏具:名詞:一般 打砕く:動詞:自立 丹:名詞:一般 つく:動詞:自立 金銀:名詞:一般 箔:名詞:一般 つく:動詞:自立 する:動詞:自立 木:名詞:一般 路:名詞:一般 ば:名詞:一般 つむ:動詞:自立 重ねる:動詞:自立 薪:名詞:一般 料:名詞:一般 売る:動詞:自立 いる:動詞:非自立 云う:動詞:自立 事:名詞:非自立 洛中:名詞:一般 始末:名詞:サ変接続 羅生門:名詞:固有名詞 修理:名詞:サ変接続 元:名詞:一般 誰:名詞:代名詞 捨てる:動詞:自立 *:名詞:一般 者:名詞:非自立 ない:形容詞:自立 荒れ果てる:動詞:自立 の:名詞:非自立 よい:形容詞:自立 

続いて、各`*.txt-decomp.txt`中に現れる単語（形容詞・動詞・名詞・副詞）について、その出現頻度をカウントします。

In [13]:
 # 実際の40作品にて、名詞・形容詞・動詞・副詞の出現回数を計数してhgramにまとめる
from collections import Counter

path = "data/"
akutagawa = ["rashomon.txt","yoba.txt","yabuno_naka.txt","mujina.txt","hana.txt",
            "haguruma.txt","torokko.txt","toshishun.txt","shunkan.txt","shujuno_kotoba.txt",
            "jashumon.txt","shogun.txt","shigo.txt","agunino_kami.txt","aruhino_oishi_kuranosuke.txt",
            "ogin.txt","ojigi.txt","kappa.txt","kiseru.txt","kumono_ito.txt"]
kikuchi = ["anegawano_kassen.txt","aru_koino_hanashi.txt","irefuda.txt","emukoshakuto_shashinshi.txt",
          "okujono_kyojin.txt","onshuno_kanatani.txt","onnagoto.txt","on'o_kaesu_hanashi.txt",
          "gimin_jinbee.txt","kunshoo_morau_hanashi.txt","gokuraku.txt","shusse.txt",
          "shobugoto.txt","dairiki_monogatari.txt","tojurono_koi.txt","aru_kogisho.txt",
          "minage_kyujogyo.txt","mumeisakkano_nikki.txt","wakasugi_saibancho.txt","bugyoto_ninsogaku.txt"]
akutagawa_name = ['羅生門', '妖婆', '藪の中', '貉', '鼻', '歯車', 'トロッコ', '杜子春', '俊寛', '侏儒の言葉', '邪宗門', '将軍', '死後', 'アグニの神', '或る日の大石内蔵助', 'おぎん', 'お時儀', '河童', '煙管', '蜘蛛の糸']
kikuchi_name = ['姉川の戦い', 'ある恋の話', '入れ札', 'M公爵と写真師', '屋上の狂人', '恩讐の彼方に', '女強盗', '恩を返す話', '義民甚平', '勲章を貰う話', '極楽', '出世', '勝負事', '大力の物語', '藤十郎の恋', 'ある抗議書', '身投げ救助業', '無名作家の日記', '若杉裁判長', '奉行と人相学']

hgram = []

for ds in akutagawa:
    counter = Counter(open(path+ds+"-decomp.txt", "r").read().split(" ")) # split(" ")ではスペースごとに区切ってリストを作成している
    hgram.append(dict(counter.most_common()))

for ds in kikuchi:
    counter = Counter(open(path+ds+"-decomp.txt", "r").read().split(" "))
    hgram.append(dict(counter.most_common()))
list(hgram[0].items())[:10]
 # hgram[i]は番号iの作品の形態素の種類とその形態素が番号iの作品にいくつ入っているかを示す辞書型オブジェクト
    
# 40編全ての名詞・形容詞・形容動詞・副詞の種類の総数
union = set()
for d in hgram:
    union |= d.keys()
print('40編の作品に現れる語の総数は', len(union))
words = list(union)

np.save('data/akutagawa_name', akutagawa_name)
np.save('data/kikuchi_name', kikuchi_name)
np.save('data/hgram', hgram)
np.save('data/words', words)


40編の作品に現れる語の総数は 16307


40編全ての作品にわたって現れる、名詞・形容詞・形容動詞・副詞の種類の総数を求めます。

In [14]:
# 40編全ての名詞・形容詞・形容動詞・副詞の種類の総数
union = set()
for d in hgram:
    union |= d.keys()
print('40編の作品に現れる語の総数は', len(union))
words = np.array(union)

40編の作品に現れる語の総数は 16307


In [15]:
akutagawa_name = np.load('data/akutagawa_name.npy', allow_pickle=True)
kikuchi_name = np.load('data/kikuchi_name.npy', allow_pickle=True)
hgram = np.load('data/hgram.npy', allow_pickle=True)
words = np.load('data/words.npy', allow_pickle=True)
print(words)
print('40編の作品に現れる語の総数は', len(words))


['たび:名詞:非自立' '縄:名詞:一般' '並べて:副詞:一般' ... '一番:名詞:副詞可能' '剣:名詞:一般' '然るに:副詞:一般']
40編の作品に現れる語の総数は 16307


特徴抽出した結果、各データは16,301次元のベクトルとなり、表形式で表すと、
40行16,301列の巨大な表となります。

今回はスペースの都合上試しに8列分だけ表示してみましょう。

> 1. 'する:動詞:自立'
1. 'いる:動詞:非自立'
1. 'の:名詞:非自立'
1. '事:名詞:非自立'
1. '云う:動詞:自立'
1. '老婆:名詞:一般'
1. 'よう:名詞:非自立'
1. 'なる:動詞:自立'

In [17]:
 # (表形式のデータの作成)
import pandas as pd

columns = ['する:動詞:自立', 'いる:動詞:非自立', 'の:名詞:非自立', 
           '事:名詞:非自立',
           '云う:動詞:自立',
           '老婆:名詞:一般',
           'よう:名詞:非自立',
           'なる:動詞:自立']

 # hgramは二つ前のセルで作成したリスト。各作品の形態素の種類とその形態素の出現回数を持つ辞書型オブジェクトを作品の数だけ持つ。
 # 本来このような関数の作成方法は良くないことに注意
 # countはn番目の作品にwという形態素は何回出現しているかを返す関数
def count(n, w):
    if w in hgram[n]:
        return hgram[n][w]
    else:
        return 0

body = []
for n in range(20):
    temp = [count(n, w) for w in columns]
    temp.append('芥川')
    body.append(temp)
for n in range(20):
    temp = [count(n+20, w) for w in columns]
    temp.append('菊池')
    body.append(temp)

df = pd.DataFrame(body)
columns.append('著者')
df.columns = columns
df.index = list(akutagawa_name) + list(kikuchi_name)
df

,する:動詞:自立,いる:動詞:非自立,の:名詞:非自立,事:名詞:非自立,云う:動詞:自立,老婆:名詞:一般,よう:名詞:非自立,なる:動詞:自立,著者
羅生門,66,53,29,28,28,28,26,15,芥川
妖婆,276,213,270,95,244,0,149,114,芥川
藪の中,42,49,59,21,30,0,20,20,芥川
貉,17,13,14,14,17,0,8,9,芥川
鼻,67,45,42,29,33,0,35,29,芥川
歯車,243,3,107,0,0,1,1,50,芥川
トロッコ,24,30,14,19,10,0,12,21,芥川
杜子春,61,79,56,0,0,0,25,31,芥川
俊寛,80,94,116,61,89,0,42,69,芥川
侏儒の言葉,402,283,194,7,108,0,62,54,芥川


以上でテキストデータから特徴抽出を行うことができました。

### hgramの保存
2.4節で作成した```hgram```は作品ごとに出現する単語とその出現回数を辞書型オブジェクトとして持つリストであり、3章や4章でも重要となります。今後、好きな時に呼び出せるように保存しておきます。

In [ ]:
np.save('')

In [ ]:
import numpy as np
np.save('hgram.txt',hgram)
!ls

In [ ]:
hgram_new = np.load('hgram.txt.npy', allow_pickle=True)